In [98]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
from dataset import CircularLearningRate

from tensorflow import config
assert config.list_physical_devices('GPU')

In [148]:
BATCH_SIZE = 32
MAXLEN = 64

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)
KINDS_SIZE = len(dataset.KINDS)

def build_model(EMBED_DIM=110, UNITS=220):

    layer = input_text = keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    layer = layers.Embedding(LETTERS_SIZE, EMBED_DIM, input_length=MAXLEN, mask_zero=True)(layer)
    layer = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(layer)
    layer = layers.add([layer, 
            layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')(layer)])
    
    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=[input_text], outputs=outputs)

    jsmodel = model 
    # keras.utils.plot_model(model, to_file='model.png')
    return model, jsmodel

model, jsmodel = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 64, 110)      4840        input_9[0][0]                    
__________________________________________________________________________________________________
bidirectional_16 (Bidirectional (None, 64, 220)      582560      embedding_8[0][0]                
__________________________________________________________________________________________________
bidirectional_17 (Bidirectional (None, 64, 220)      776160      bidirectional_16[0][0]           
____________________________________________________________________________________________

In [100]:
def fit(train_validation, scheduler, verbose=1):
    train, valid = train_validation
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    callbacks = []
    if isinstance(scheduler, CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin, 'C': train.normalized }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin, 'C': valid.normalized }
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


In [101]:
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [142]:
data_other = load_data(['biblical', 'garbage', 'poetry'])

In [143]:
data_rabanit = load_data(['rabanit'])

In [144]:
data_pre_modern = load_data(['pre_modern'])

In [145]:
data_modern = load_data(validation=0.2, source=['modern'])

In [ ]:
model.load_weights('./checkpoints/uninit')
history = fit(data_other, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))  # val_N_accuracy: 0.9201 - val_D_accuracy: 0.9736
model.save_weights('./checkpoints/other')

Train on 49673 samples, validate on 90 samples
10848/49673 [=====>........................] - ETA: 1:13 - loss: 1.0255 - N_loss: 0.8171 - D_loss: 0.1648 - S_loss: 0.0436 - N_accuracy: 0.7138 - D_accuracy: 0.9351 - S_accuracy: 0.9880

In [ ]:
model.load_weights('./checkpoints/other')
history = fit(data_rabanit, scheduler=CircularLearningRate(20e-4, 50e-4, 5e-4))  # val_N_accuracy: 0.9301 - val_D_accuracy: 0.9787
model.save_weights('./checkpoints/rabanit')

In [ ]:
model.load_weights('./checkpoints/rabanit')
history = fit(data_pre_modern, scheduler=CircularLearningRate(20e-4, 40e-4, 0.1e-4))  # val_N_accuracy: 0.9349 - val_D_accuracy: 0.9803
model.save_weights('./checkpoints/pre_modern')

In [ ]:
model.load_weights('./checkpoints/pre_modern')
history = fit(data_modern, scheduler=CircularLearningRate(6e-3, 6e-3, 0.1e-3))  #  val_N_accuracy: 0.9609 - val_D_accuracy: 0.9853
# history = fit(data_modern, scheduler=CircularLearningRate(1e-3, 7e-3, 0.5e-3))
model.save_weights('./checkpoints/modern')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
jsmodel.summary()

In [87]:
import tensorflowjs as tfjs
jsmodel.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(jsmodel, '.')

In [140]:
model.load_weights('./checkpoints/modern')

def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)

print_predictions(data_modern[1], 10)

מצוי:  מִלְחָמָה - יֵיאָלְצוּ אוֹתָנוּ לְכָךְ. אִם נַעֲשָׂה זֹאת נַחְסוֹךְ לְעַצְמֵנוּ שָׁנִים שֶׁל 
רצוי:  מִלְחָמָה - יֵיאָלְצוּ אוֹתָנוּ לְכָךְ. אִם נַעֲשָׂה זֹאת נַחֲסוֹךְ לְעַצְמֵנוּ שָׁנִים שֶׁל 
מצוי:  י"ג-י"ד, וְהֵן בְּעִיקָּר תְּעוּדוֹת הוֹרָאָה, תְּעוּדוֹת הַכְשָׁרָה מִקְצוֹעִית 
רצוי:  י"ג-י"ד, וְהֵן בְּעִיקָּר תְּעוּדוֹת הוֹרָאָה, תְּעוּדוֹת הַכְשָׁרָה מִקְצוֹעִית 
מצוי:  הַזַּעֲזוּעַ מֵהָרֶצַח סַחַף אַחֲרָיו רָצוֹן לִפְעוֹל פְּעוּלָּה מַעֲשִׂית לְעִיצּוּב פְּנֵי הַחֶבְרָה 
רצוי:  הַזַּעֲזוּעַ מֵהָרֶצַח סָחַף אַחֲרָיו רָצוֹן לִפְעוֹל פְּעוּלָּה מַעֲשִׂית לְעִיצּוּב פְּנֵי הַחֶבְרָה 
מצוי:  שֶׁל הַחַלַת רִיבּוֹנוּת יִשְׂרְאֵלִית בַּשְׁטָחִים מֵעֵבֶר לְקַו הַיָּרוֹק מַעְלָה שְׁאֵלוֹת 
רצוי:  שֶׁל הַחַלַּת רִיבּוֹנוּת יִשְׂרְאֵלִית בַּשְּׁטָחִים מֵעֵבֶר לַקַּו הַיָּרוֹק מַעֲלֶה שְׁאֵלוֹת 
מצוי:  קְצָת יוֹתֵר. חֶבְרָתִי פְּרְגָנָה לִשְׁנֵינוּ, וּמֵאָז וְעַד הַיּוֹם אֲנַחְנוּ בְּנֵי זוּג, 
רצוי:  קְצָת יוֹתֵר. חֲבֶרְתִּי פִרְגְּנָה לִשְׁנֵינוּ, וּמֵאָז וְעַד הַיּוֹם אֲנַחְנוּ בּ

In [ ]:
shutil.rmtree(os.sep.join([tempfile.gettempdir(), '.tensorboard-info']), ignore_errors=True)
shutil.rmtree('logs', ignore_errors=True)
os.makedirs('logs')
# %tensorboard --logdir logs

In [ ]:
print(data_modern[1].text[0])
print(data_modern[1].text[1])